In [1]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, LSTM, Flatten, Reshape
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
from tensorflow.keras import layers, models
import gc
import tensorflow as tf
import random
from sklearn import preprocessing
import pickle
import tqdm
from utilities import *

In [2]:
# Load config
config = loadData("config")
dataSetSize = config["dataSetSize"]
testSetBenignSize = config["testSetBenignSize"]
validationSetSize = config["validationSetSize"]
trainingSetSize = config["trainingSetSize"]
sequenceLen = config["sequenceLen"]
dimensionsCount = config["dimensionsCount"]
numberOfAttackSamplesToChoose = 100
testSetSize = testSetBenignSize + numberOfAttackSamplesToChoose

In [12]:
# Load thresholds
thresholds = loadData("thresholds")

In [4]:
def seqToFuncModel(model):
    input_layer = layers.Input(batch_shape=model.layers[0].input_shape)
    prev_layer = input_layer
    for layer in model.layers:
        prev_layer = layer(prev_layer)

    funcModel = models.Model([input_layer], [prev_layer])
    return funcModel

In [5]:
@tf.function # To fix: 'Tensor' object has no attribute 'numpy' 
def create_adversarial_pattern(model, input_image, input_label):
    funcModel = seqToFuncModel(model)
    loss_object = tf.keras.losses.MeanSquaredError()
    with tf.GradientTape() as tape:
        tape.watch(input_image)
        prediction = funcModel(input_image)
        #loss = loss_object(input_label, prediction)
        loss = loss_object(input_image, prediction)

    # Get the gradients of the loss w.r.t to the input image.
    gradient = tape.gradient(loss, input_image)
    # Get the sign of the gradients to create the perturbation
    signed_grad = tf.sign(gradient)
    return signed_grad

In [6]:
@tf.function 
def mzAttackSampleGeneratorFromBenign(model, iterations,seed,epsilon):
    tfInput = tfoutput = seed
    for step in range(0,iterations):
        tfInput = seed
        tfOutput = tfInput
        sgrad = create_adversarial_pattern(model, tfInput, tfOutput)                
        epsilon = 0.001
        perturbations = epsilon*sgrad
        seed = seed + perturbations    
    attackSample = seed
    return attackSample

In [7]:
@tf.function 
def mzAttackSampleGeneratorFromAnomaly(model, iterations,seed,epsilon):
    tfInput = tfoutput = seed
    for step in range(0,iterations):
        tfInput = seed
        tfOutput = tfInput
        sgrad = create_adversarial_pattern(model, tfInput, tfOutput)                        
        perturbations = epsilon*sgrad
        seed = seed - perturbations 
        
    attackSample = seed    
    return attackSample

In [8]:
def generateAttackSamples(model, modelType, count):
    attackSamples = list()    
    for i in tqdm.tqdm(range(count)):        
        randSample = getReshapedTestSet(np.random.rand(1,sequenceLen,dimensionsCount),modelType)        
        mse = 1000
        attackSample = randSample
        while mse > thresholds[modelType]:            
            attackSample = mzAttackSampleGeneratorFromAnomaly(model, 1, attackSample, 0.001)            
            predicted = model.predict(attackSample)
            if(modelType == "LSTM"):
                mse = (np.square(attackSample - predicted)).mean(axis=2).mean(axis=1)  
            else:
                mse = (np.square(attackSample - predicted)).mean(axis=1) 
        attackSamples.append(attackSample)
    return attackSamples

### TMP PCA SECOND TIME GENERATION

In [10]:
def generateAttackSamplesAdv(model, modelType, count):
    attackSamples = list()    
    for i in tqdm.tqdm(range(count)):        
        randSample = getReshapedTestSet(np.random.rand(1,sequenceLen,dimensionsCount),modelType)        
        mse = 1000
        attackSample = randSample
        while mse > thresholds[modelType + "_adv"]:            
            attackSample = mzAttackSampleGeneratorFromAnomaly(model, 1, attackSample, 0.001)            
            predicted = model.predict(attackSample)
            if(modelType == "LSTM"):
                mse = (np.square(attackSample - predicted)).mean(axis=2).mean(axis=1)  
            else:
                mse = (np.square(attackSample - predicted)).mean(axis=1) 
        attackSamples.append(attackSample)
    return attackSamples

In [13]:
model = load_model('Trained_Model/pca_adv.h5')
attackSamples = generateAttackSamplesAdv(model, "PCA",100)
dataArray = np.array(attackSamples)
saveData(dataArray, "adversarial_data_set_pca_after_adv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [19:56<00:00, 11.97s/it]


### PCA

In [20]:
model = load_model('Trained_Model/pca.h5')
attackSamples = generateAttackSamples(model, "PCA",500)
dataArray = np.array(attackSamples)
saveData(dataArray, "adversarial_data_set_pca")

100%|██████████████████████████████████████████████████████████████████████████████| 500/500 [1:52:08<00:00, 13.46s/it]


### Fully Connected

In [22]:
model = load_model('Trained_Model/autoencoder.h5')
attackSamples = generateAttackSamples(model, "fullyConnected",500)
dataArray = np.array(attackSamples)
saveData(dataArray, "adversarial_data_set_fullyConnected")

100%|██████████████████████████████████████████████████████████████████████████████| 500/500 [1:50:41<00:00, 13.28s/it]


### 1D Conv

In [23]:
model = load_model('Trained_Model/conv.h5')
attackSamples = generateAttackSamples(model, "1DConv",500)
dataArray = np.array(attackSamples)
saveData(dataArray, "adversarial_data_set_conv")

100%|██████████████████████████████████████████████████████████████████████████████| 500/500 [3:34:40<00:00, 25.76s/it]


### LSTM

In [9]:
## THIS CELL IS NOT WORKING -> GO TO THE NEXT ONE
#model = load_model('Trained_Model/lstm.h5')
#attackSamples = generateAttackSamples(model, "LSTM",500)
#dataArray = np.array(attackSamples)
#saveData(dataArray, "adversarial_data_set_lstm")

### For LSTM, the above code won't work but this works

In [11]:
model = load_model('Trained_Model/lstm.h5')

# Convert sequential to functional model
from tensorflow.keras import layers, models

input_layer = layers.Input(batch_shape=model.layers[0].input_shape)
prev_layer = input_layer
for layer in model.layers:
    prev_layer = layer(prev_layer)

funcmodel = models.Model([input_layer], [prev_layer])

@tf.function # To fix: 'Tensor' object has no attribute 'numpy' 
def create_adversarial_pattern_lstm(input_image, input_label):
    loss_object = tf.keras.losses.MeanSquaredError()
    with tf.GradientTape() as tape:
        tape.watch(input_image)
        prediction = funcmodel(input_image)
        #loss = loss_object(input_label, prediction)
        loss = loss_object(input_image, prediction)

    # Get the gradients of the loss w.r.t to the input image.
    gradient = tape.gradient(loss, input_image)
    # Get the sign of the gradients to create the perturbation
    signed_grad = tf.sign(gradient)
    return signed_grad

@tf.function 
def mzAttackSampleGeneratorFromZero(iterations,seed,epsilon):
    tfInput = tfoutput = seed
    for step in range(0,iterations):
        tfInput = seed
        tfOutput = tfInput
        sgrad = create_adversarial_pattern_lstm(tfInput, tfOutput)                        
        perturbations = epsilon*sgrad
        seed = seed - perturbations    
        #print(perturbations.shape)
        #print(adv_x.shape)    
        #plt.plot(adv_x[0,:,6])
        #plt.figure()    
    attackSample = seed
    return attackSample



attackSamples = list()
# Generating attack samples from random seeds
for i in tqdm.tqdm(range(500)):
    randSample = np.random.rand(1,sequenceLen,dimensionsCount)
    mse = 1000
    attackSample = randSample
    while mse > thresholds["LSTM"]:
        attackSample = mzAttackSampleGeneratorFromZero(1,attackSample,0.001)
        predicted = model.predict(attackSample)
        mse = (np.square(attackSample - predicted)).mean(axis=2).mean(axis=1)        
    attackSamples.append(attackSample)

100%|████████████████████████████████████████████████████████████████████████████| 500/500 [16:07:20<00:00, 116.08s/it]


In [12]:
dataArray = np.array(attackSamples)
saveData(dataArray, "adversarial_data_set_lstm")